In [1]:
import pandas as pd
import json
import pdb
from datetime import date, datetime
import re
import numpy as np
import os

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [80]:
file_schema = open('../../../../Downloads/cdl_assam_tenders_ocds_template.json',encoding='utf-8', errors='ignore')
data = json.load(file_schema)

In [81]:
awarded_df = pd.DataFrame(columns = ['Bid Number', 'Bidder Name', 'Awarded Currency', 'Awarded Value',
       'Contract Date :', 'Contract Value :'])
df_bidders = pd.DataFrame(columns =['S.No.1', 'Bid Number', 'Bidder Name', 'Submitted Date', 'Status',
       'Remarks'])

In [82]:
def add_columns(df):
    df['fiscal_year'] = pd.to_datetime(df['Publish Date']).dt.to_period('Q-APR').dt.qyear.apply(lambda x: str(x-1) + "-" + str(x))
    df['ocid'] = "ocds-f5kvwu" + "-" + df['Tender ID']
    df['initiationType'] = "tender"
    df['tag'] = "tender"
    df['id'] =  str(1)
    df['date'] = date.today()

In [83]:
def format_money_values(value):
    pattern = r"[-+]?\d*\.\d+|\d+"
    matches = re.findall(pattern,str(value))
    result = ''.join(matches)
    if result == '':
        return result
    else:
        return float(result)

In [84]:
def time_difference(str_1,str_2):
    # Define the two date strings
    date_str1 = str_1
    date_str2 = str_2

    # Define the format of the date strings
    date_format = "%d-%b-%Y %I:%M %p"

    # Convert the strings to datetime objects
    datetime_obj1 = datetime.strptime(date_str1, date_format)
    datetime_obj2 = datetime.strptime(date_str2, date_format)

    # Calculate the time difference
    time_difference = datetime_obj1 - datetime_obj2

    # Extract the days, seconds, and total seconds from the timedelta
    days = time_difference.days

    # Format the result as a string
    formatted_delta = days

    return formatted_delta


In [85]:
def format_date(date):
    if date == "NA" or date == "NA ":
        return date
    date_str = date.strip()
    
    # Define the format of the date strings
    date_format_1 = "%d-%b-%Y %I:%M %p"
    date_format_2 = "%Y-%d-%m %H:%M"

    # Convert the strings to datetime objects
    datetime_obj1 = datetime.strptime(date_str, date_format_1)
    datetime_obj1 = datetime_obj1.strftime(date_format_2)
    return datetime_obj1

In [86]:
def format_date_contract(date):
    date_str = date.strip()
    
    # Define the format of the date strings
    date_format_1 = "%d-%b-%Y"
    date_format_2 = "%Y-%d-%m"

    # Convert the strings to datetime objects
    datetime_obj1 = datetime.strptime(date_str, date_format_1)
    datetime_obj1 = datetime_obj1.strftime(date_format_2)
    return datetime_obj1

In [87]:
def format_date_milestone(date):
    date_str = date.strip()

    # Define the format of the date strings
    date_format_1 = "%d-%b-%Y %I:%M %p"
    date_format_2 = "%Y-%d-%m %H:%M"

    # Convert the strings to datetime objects
    datetime_obj1 = datetime.strptime(date_str, date_format_1)
    datetime_obj1 = datetime_obj1.strftime(date_format_2)
    return datetime_obj1

In [88]:
def add_extra_info_json(df,data):
    data['ocid'] = df['ocid'][0]
    data['id'] =  str(1)
    data['date'] = date.today()
    data['tag'] = [df['tag'][0]]
    data['initiationType'] = "tender"

In [89]:
def extract_individual_dataframes(dataframe,df,column_name,index_of_column,name_of_new_columns):
    dataframe_name = dataframe
    if column_name in df.columns:
        column_index = df.columns.get_loc(column_name)
        # Select the "Submitted Date" column and 3 columns before and after it
        columns_to_select = df.iloc[:, column_index - index_of_column:column_index + index_of_column]

        dataframe = columns_to_select
        dataframe= dataframe.dropna(how = 'all')
        dataframe.columns = name_of_new_columns
    else:
        dataframe = pd.DataFrame(columns = name_of_new_columns)
    if "Submitted Date" in name_of_new_columns:
        dataframe = dataframe.dropna(subset = ['Bidder Name'])
        dataframe = dataframe[dataframe['Bidder Name'] != 'na']
        dataframe.reset_index(inplace=True)
    if "Awarded Value" in name_of_new_columns:
        dataframe = dataframe.dropna(subset = ['Awarded Value'])
    return dataframe

In [90]:
def add_parties(data,df,df_bidders,awarded_df):
    del data['parties'][:-1]
    data['parties'][0]['id'] = re.sub('\W+','', df['Organisation Chain'][0].split("|")[0]).lower()
    data['parties'][0]['name'] = df['Organisation Chain'][0].split("|")[0]
    data['parties'][0]['roles'] = ["buyer"]
    if not df_bidders.empty:
        for i in df_bidders['Bidder Name']:
            if i in awarded_df['Bidder Name'].tolist():
                data['parties'].append({'id':  re.sub('\W+','', str(i) ).lower(),'name': i,'roles': ['tenderer','supplier']})
            else:
                data['parties'].append({'id':  re.sub('\W+','', str(i) ).lower(),'name': i,'roles': ['tenderer']})

#         for i in df_bidders['Bid Number']:
#              data['parties'].append({'id': i,'name': i,'roles': ['tenderer']})
    data['parties'].append({'id': re.sub('\W+','', df['Name'][0] ).lower(),'name':df['Name'][0],'address':{'streetAddress':df['Address'][0]},'roles':['procuringEntity']})
#     data['parties'] = pd.Series(data['parties']).to_dict()

In [91]:
def add_buyer(data,df):
    data['buyer']['name'] =   df['Organisation Chain'][0].split("|")[0]
    data['buyer']['id'] = re.sub('\W+','', df['Organisation Chain'][0].split("|")[0]).lower()

In [92]:
def check_cols(df,column_list):
    new_list =list(set(df.columns).union(column_list))
    df=df.reindex(columns=new_list).fillna("NA")
    return df

In [93]:
def add_tender_info(data,df):
    data['tender']['id'] = df['Tender ID'][0]
#     data['tender']['externalReference'] = df['Tender Reference Number'][0]
    data['tender']["identifiers"] =  [{"id": str(df['Tender Reference Number'][0]),"scheme": "external"}]
    data['tender']['title'] = df['Title'][0]
    data['tender']['description'] = df['Work Description'][0]
    data['tender']["datePublished"] = format_date(df['Publish Date'][0])
    data['tender']["contractType"] = df['Contract Type'][0]
    data['tender']["submissionTerms"] = {}
    data['tender']["submissionTerms"]["allowPreferentialBidder"]=df['Allow Preferential Bidder'][0]
    data['tender']["submissionTerms"]["allowMultiCurrencyFee"]= df["Is Multi Currency Allowed For Fee"][0]
    data['tender']["submissionTerms"]["allowMultiCurrencyBQQ"] = df['Is Multi Currency Allowed For BOQ'][0]
    data['tender']["submissionTerms"]['bidValidityPeriod'] =  {'durationInDays': int(df['Bid Validity(Days)'][0])}
    data['tender']["procuringEntity"]["name"]=df['Name'][0]
    data['tender']['procuringEntity']['id'] = re.sub('\W+','', df['Name'][0]).lower()
    data['tender']['items'][0]['id'] = "01"
    data['tender']['items'][0]['classification']['description'] = df['Product Category'][0]
    if df['Sub category'][0] == "NA " or df['Sub category'][0] == "NA":
        del data['tender']['items'][0]['additionalClassifications']['description']
    else:
#         data['tender']['items'][0]['additionalClassifications']['description'] = df['Sub category'][0]
        data['tender']['items'][0]['additionalClassifications'] = [{'description': df['Sub category'][0]}]
    data['tender']['items'][0]['deliveryAddresses'][0]['postalCode'] = str(df['Pincode'][0])
    data['tender']['items'][0]['deliveryAddresses'][0]['streetAddress'] = df['Location'][0]

    if (type(df['Tender Value in ₹'][0]) == float) or (str(df['Tender Value in ₹'][0]) == "nan") :
        data['tender']['value']["amount"] = df['Tender Value in ₹'][0]
        data['tender']['value']["currency"] = "INR"
    elif df['Tender Value in ₹'][0] == 0.0:
        data['tender']['value']["amount"] = 0
        data['tender']['value']["currency"] = "INR"
    elif df['Tender Value in ₹'][0] == "NA":
        del data['tender']['value']
    else:
        data['tender']['value']["amount"] = format_money_values(df['Tender Value in ₹'][0])
        data['tender']['value']["currency"] = "INR"
    data['tender']['procurementMethod'] = df['Tender Type'][0]
    data['tender']["mainProcurementCategory"] = df['Form of contract'][0] 
    data['tender']['tenderPeriod']['startDate'] = format_date(df['Publish Date'][0])
    data['tender']['tenderPeriod']['endDate'] = format_date(df['Bid Submission End Date'][0])
    data['tender']['tenderPeriod']['durationInDays'] = time_difference(df['Bid Submission End Date'][0],df['Publish Date'][0])
    data['tender']['enquiryPeriod']['startDate'] = format_date(df['Clarification Start Date'][0])
    data['tender']['enquiryPeriod']['endDate'] = format_date(df['Clarification End Date'][0])
    if df['Clarification End Date'][0] == 'NA ' or df['Clarification Start Date'][0] == "NA" or df['Clarification End Date'][0] == 'NA' or df['Clarification Start Date'][0] == 'NA ':
        data['tender']['enquiryPeriod']['durationInDays'] = "NA"
    else:
        data['tender']['enquiryPeriod']['durationInDays'] = time_difference(df['Clarification End Date'][0].strip(), df['Clarification Start Date'][0].strip())
#     data['tender']['contractPeriod'] = df['Period Of Work(Days)'][0]
    try:
        data['tender']['contractPeriod'] = {"durationInDays":int(df['Period Of Work(Days)'][0])}
    except:
        data['tender']['contractPeriod'] = {"durationInDays":df['Period Of Work(Days)'][0]}
    data['tender']['communication']['documentAvailabilityPeriod']['startDate'] = format_date(df['Document Download / Sale Start Date'][0])
    data['tender']['communication']['documentAvailabilityPeriod']['endDate'] = format_date(df['Document Download / Sale End Date'][0])
    data['tender']['bidOpening']['date'] = format_date(df['Bid Opening Date'][0])
    data['tender']['bidOpening']['address']["streetAddress"] = df['Bid Opening Place'][0]
    data['tender']['participationFees'][0]['id'] = "01"
    if type(df['Tender Fee in ₹'][0]) == float:
        data['tender']['participationFees'][0]['value']['amount'] = df['Tender Fee in ₹'][0]
    elif df['Tender Fee in ₹'][0] == 0.0:
        data['tender']['participationFees'][0]['value']['amount'] = 0
    else:
        data['tender']['participationFees'][0]['value']['amount'] = format_money_values(df['Tender Fee in ₹'][0])
    data['tender']['participationFees'][0]['value']['currency'] = "INR"
    data['tender']['participationFees'][0]['description'] = df['Fee Payable To'][0]
    if df['Bid Number'][0] != "NA":
        data['tender']['numberOfTenderers'] = int(df['Bid Number'].count())
    else:
        del data['tender']['numberOfTenderers']
    if not df_bidders.empty:
        for i in df_bidders['Bidder Name']:
               data['tender']['tenderers'].append({'name': i,
                   'id':  re.sub('\W+','', i ).lower()})
        data['tender']['tenderers'].pop(0)

    else:
        del data['tender']['tenderers']
#         data['tender']['tenderers'][0]['name'] = "NA"
#         data['tender']['tenderers'][0]['id'] = "NA"
    del data['tender']['milestones'][:-1]
    data['tender']['milestones'].append({'id': "publisheddate",
      'title': 'Published Date',
      'type': 'Pre-procurement milestones',
      'dueDate': format_date_milestone(df['Publish Date'][0])})
    try:
        data['tender']['milestones'].append({'id': "documentdownloadsalestartdate",
            'title': 'Document Download / Sale Start Date',
            'type': 'Pre-procurement milestones',
            'dueDate': format_date_milestone(df['Document Download / Sale Start Date'][0])})
    except:
        print("NA")

    try:
        data['tender']['milestones'].append({'id': "clarificationstartdate",
              'title': 'Clarification Start Date',
              'type': 'Pre-procurement milestones',
              'dueDate': format_date_milestone(df['Clarification Start Date'][0])})
    except:
        print("NA")
    try:
        data['tender']['milestones'].append({'id': "bidsubmissionstartdate",
              'title': 'Bid Submission Start Date',
              'type': 'Pre-procurement milestones',
              'dueDate': format_date_milestone(df['Bid Submission Start Date'][0])})
    except:
        print("NA")
    try:
        data['tender']['milestones'].append({'id': "bidopeningdate",
          'title': 'Bid Opening Date',
          'type': 'Pre-procurement milestones',
          'dueDate': format_date_milestone(df['Bid Opening Date'][0])})
    except:
        print("NA")
    try:
        data['tender']['milestones'].append({'id': "documentdownloadsaleenddate",
          'title': 'Document Download / Sale End Date',
          'type': 'Pre-procurement milestones',
          'dueDate': format_date_milestone(df['Document Download / Sale End Date'][0])})
    except:
        print("NA")
    try:
        data['tender']['milestones'].append({'id': "clarificationenddate",
          'title': 'Clarification End Date',
          'type': 'Pre-procurement milestones',
          'dueDate': format_date_milestone(df['Clarification End Date'][0])})
    except:
        print("NA")
    try:
        data['tender']['milestones'].append({'id': "bidsubmissionenddate",
          'title': 'Bid Submission End Date',
          'type': 'Pre-procurement milestones',
          'dueDate': format_date_milestone(df['Bid Submission End Date'][0])})
    except:
        print("NA")
    
    try:
        data['tender']['milestones'].append({'id':"financialbidopeningdate",
                                            'title':'Financial Bid Opening Date',
                                            'type': 'Pre-procurement milestones',
                                            'dueDate': format_date_milestone(df['Financial Bid Opening Date'][0])})
    except:
        print("NA")
    try:
        data['tender']['milestones'].append({'id':'contractdate',
                                            'title':'Contract Date',
                                            'type':'Pre-procurement milestones',
                                            'dueDate':format_date_milestone(df['Contract Date :'][0])})
    except:
        print("NA")
    for i in range(len(data['tender']['milestones'])):
        data['tender']['milestones'][i]['id'] = str(i)
    if df["Published Date"][0] != "NA":
        data['tender']['amendments'][0]['date'] = format_date(df['Published Date'][0])
        data['tender']['amendments'][0]['rationale'] = df["Corrigendum Reason"][0]
        data['tender']['amendments'][0]['description'] = df["Corrigendum Description"][0]
    else:
        del data['tender']['amendments']
    data['tender']['milestones'].pop(0)
    data['tender']['fiscalYear'] = df['fiscal_year'][0]

In [94]:
def add_awards(data,df,awarded_df):
    if df['Contract Value :'][0] == "NA":
        del data['awards']
        return
    del data['awards']
    data['awards'] = []
    if df['Awarded Value'][0] != "NA":
        for i,x in enumerate(awarded_df['Bidder Name']):
            if type(x) == float:
                    data['awards'].append({
                          "id": str(i),
                          "value": {
                            "amount": format_money_values(awarded_df['Awarded Value'][i]),
                            "currency": awarded_df['Awarded Currency'][i]
                          },
                          "suppliers": [
                            {
                              "id": str(x),
                              "name": x
                            }
                          ],
                'contractPeriod': {'startDate': format_date_contract(df['Contract Date :'][0]),
               'durationInDays':  int(df['Work Completion Period in days :'][0])}
                        })
            else:
                data['awards'].append({
                    "id": str(i),
                      "value": {
                        "amount": format_money_values(awarded_df['Awarded Value'][i]),
                        "currency": awarded_df['Awarded Currency'][i]
                      },
                      "suppliers": [
                        {
                          "id": re.sub('\W+','',x).lower(),
                          "name": x
                        }
                      ],
                   'contractPeriod': {'startDate': format_date_contract(df['Contract Date :'][0]),
               'durationInDays':  int(df['Work Completion Period in days :'][0])}
                    })
#     try:
#         data['awards'].append({'contractPeriod': {'startDate': format_date_contract(df['Contract Date :'][0]),
#                'durationInDays':  int(df['Work Completion Period in days :'][0])}})
#     except:
#         data['awards'].append({'contractPeriod': {'startDate': format_date_contract(df['Contract Date'][0]),
#            'durationInDays': df['Work Completion Period in days :'][0]}})
#     del data['awards']
#     data['awards'] = [{'id': 'assign id here unique within the release - this could be just 01',
#      'description': 'AOC > AOC Description :',
#      'value': {'amount': 'Awarded Bids List > Awarded Value or AOC > Contract Value',
#       'currency': 'Awarded Bids List > Awarded Currency'},
#      'suppliers': [{'id': 'id assigned to this bidder in the parties array',
#        'name': 'Awarded Bids List > Bidder Name'}],
#      'contractPeriod': {'startDate': 'AOC > Contract Date',
#       'durationInDays': 'AOC Work Completion Period in days'}}]

#     if df['Contract Value :'][0] == "NA":
#         del data['awards']
#         return
#     try:
#         data['awards'].append({'id': 1,
#           'description': df['AOC Description :'][0],
#           'value': {'amount':format_money_values(df['Contract Value :'][0]),                    
# #           'value': {'amount':re.sub("[^0-9]", "", df['Contract Value :'][0]),
#            'currency': df['Awarded Currency'][0]},
#              'suppliers': [{'id': 'id assigned to this bidder in the parties array',
#            'name': 'Awarded Bids List > Bidder Name'}],
#           'contractPeriod': {'startDate': format_date_contract(df['Contract Date :'][0]),
#            'durationInDays':  int(df['Work Completion Period in days :'][0])}})
#     except:
#         data['awards'].append({'id': 1,
#           'description': df['AOC Description :'][0],
#           'value': {'amount':format_money_values(df['Contract Value'][0]),
#            'currency': df['Awarded Currency'][0]},
#           'suppliers': [{'id': 'id assigned to this bidder in the parties array',
#                'name': 'Awarded Bids List > Bidder Name'}],
#           'contractPeriod': {'startDate': format_date_contract(df['Contract Date'][0]),
#            'durationInDays': df['Work Completion Period in days :'][0]}})
#     if type(df['Awarded Value'][0]) == float:
#         data['awards'].append({'id':2,'value':{'amount':df['Awarded Value'][0],
#                                           'currency':"INR"}})
#     elif df['Awarded Value'][0] == 0.0:
#         data['awards'].append({'id':2,'value':{'amount': 0,
#                                           'currency':"INR"}})
#     else:
#         data['awards'].append({'id':2,'value':{'amount':re.sub("[^0-9]", "", str(df['Awarded Value'][0])),
#                                           'currency':"INR"}})
#     data['awards'].pop(0)
# #     data['awards'] = pd.Series(data['awards']).to_dict()
#     if df['Awarded Value'][0] != "NA":
#         for i,x in enumerate(awarded_df['Bidder Name']):
#             if type(x) == float:
#                 data['awards'][0]['suppliers'].append({'id':  x,
#               'name': x})
#             else:
#                 data['awards'][0]['suppliers'].append({'id':  re.sub('\W+','',x).lower(),
#               'name': x})
#         data['awards'][0]['suppliers'].pop(0)
#     else:
#           del data['awards']
# #         data['awards'][0]['suppliers'][0]['id']  = "NA"
# #         data['awards'][0]['suppliers'][0]['name'] = "NA"
# #     #     data['awards'][0]['suppliers'].pop(0)

In [95]:
def add_bids(data,df,df_bidders,awarded_df):
    for i,x in enumerate(df_bidders['Bid Number']):
        if df_bidders['Bid Number'][i] not in awarded_df['Bid Number'].values:
            data['bids']['details'].append({'id': str(df_bidders['Bid Number'][i]),
               'date': format_date(df_bidders['Submitted Date'][i]),
               'status': df_bidders['Status'][i],
#                'validityPeriod': {'durationInDays': df['Bid Validity(Days)'][0]},
               'tenderers': [{'id':re.sub('\W+','', str(df_bidders['Bidder Name'][i])).lower() ,
                 'name': df_bidders['Bidder Name'][i]}]})
        else:
            continue
    for i,x in enumerate(df_bidders['Bid Number']):
        if df_bidders['Bid Number'][i] in awarded_df['Bid Number'].values:
            data['bids']['details'].append({'id': str(df_bidders['Bid Number'][i]),
                       'date': format_date(df_bidders['Submitted Date'][i]),
                       'status': df_bidders['Status'][i],
#                        'validityPeriod': {'durationInDays': df['Bid Validity(Days)'][0]},
                       'tenderers': [{'id':re.sub('\W+','', str(df_bidders['Bidder Name'][i])).lower() ,
                         'name': df_bidders['Bidder Name'][i]}]})
    data['bids']['details'].pop(0)
    if len(data['bids']['details']) == 0:
           del data['bids']
#          data['bids']['details'].append({'id': "NA",
#                        'date': "NA",
#                        'status': "NA",
# #                        'validityPeriod': {'durationInDays': "NA"},
#                        'tenderers': [{'id':"NA" ,
#                          'name': "NA"}]})

In [96]:
def save_json(data,path,name):
    with open(path+name+".json","w") as out:
        json.dump(data,out,default=str,indent=4)

In [97]:
def remove_parent_keys(d, remove_all_if_all_children_na=False):
    if isinstance(d, dict):
        new_dict = {
            key: remove_parent_keys(value, remove_all_if_all_children_na)
            for key, value in d.items()
            if not (
                (remove_all_if_all_children_na and isinstance(value, dict) and all(val in ("NA", "NA ") for val in value.values())) or
                (not remove_all_if_all_children_na and value in ("NA", "NA "))
            )
        }
        if new_dict:
            return new_dict
    elif isinstance(d, list):
        return [remove_parent_keys(item, remove_all_if_all_children_na) for item in d]
    else:
        return d


In [98]:
def remove_null_values(d):
    if isinstance(d, dict):
        return {key: remove_null_values(value) for key, value in d.items() if value is not None}
    elif isinstance(d, list):
        return [remove_null_values(item) for item in d]
    else:
        return d

In [99]:
def remove_empty_arrays(d):
    if isinstance(d, dict):
        return {key: remove_empty_arrays(value) for key, value in d.items()}
    elif isinstance(d, list) and not d:
        return None  # Replace empty arrays with None
    else:
        return d

In [100]:
def add_statistics(data,df): 
    if df['Contract Value :'][0] != 'NA':
        data['statistics'] = []
        try:
             data["statistics"].append(
            {
              "id": "1",
              "value": float(re.sub(r'[^\d\.]', '', df['Contract Value :'][0])),
              "currency": "INR",
              "measure": "contractValue"
            }
             )
        except:
             data["statistics"].append(
            {
              "id": "1",
              "value": format_money_values(df['Contract Value'][0]),
              "currency": "INR",
              "measure": "contractValue"
            }
             )
    elif df['Contract Value'][0] != 'NA':
        data['statistics'] = []
        try:
             data["statistics"].append(
            {
              "id": "1",
              "value":float(re.sub(r'[^\d\.]', '', df['Contract Value :'][0])),
              "currency": "INR",
              "measure": "contractValue"
            }
             )
        except:
             data["statistics"].append(
            {
              "id": "1",
              "value": format_money_values(df['Contract Value'][0]),
              "currency": "INR",
              "measure": "contractValue"
            }
             )
    else:
        return

In [101]:
def main(df,data,awarded_df,df_bidders):
    add_columns(df)
    add_extra_info_json(df,data)
    df_bidders = extract_individual_dataframes(df_bidders,df,"Submitted Date",3,['S.No.1', 'Bid Number', 'Bidder Name', 'Submitted Date', 'Status',
       'Remarks'])
    awarded_df = extract_individual_dataframes(awarded_df,df,"Awarded Value",3,['Bid Number', 'Bidder Name', 'Awarded Currency', 'Awarded Value',
       'Contract Date :', 'Contract Value :'])
    add_parties(data,df,df_bidders,awarded_df)
    add_buyer(data,df)
    df.shape
    column_list = ['Publish Date',' Bid Number.1','Contract Value','Awarded Currency','AOC Description :', 'Contract Value :',
                'Bidder Name.1','Contract Date :','Bid Number','Awarded Value','Published Date', 'Contract Date',
                 'Corrigendum Reason','Corrigendum Description','Organisation Chain','Submitted Date',
                 'Awarded Value','Work Completion Period in days :','Financial Bid Opening Date']
    df = check_cols(df,column_list)
    add_tender_info(data,df)
    add_awards(data,df,awarded_df)
    add_bids(data,df,df_bidders,awarded_df)
    add_statistics(data,df)
    del data['tender']['documents']
    data = remove_parent_keys(data, remove_all_if_all_children_na=False)
    data  = remove_parent_keys(data, remove_all_if_all_children_na=True)
    data = remove_null_values(data)
    data = remove_empty_arrays(data)
#     save_json(data,"",'tttteeesst')
    save_json(data,"jsons-v10/",df['Tender ID'][0])

In [ ]:
dir_path = 'concatinated_csvs/'
file_extension = '.csv'
file_list = []
for file in os.listdir(dir_path):
    if file.endswith(file_extension):
        file_list.append(file)
for file in file_list:
    tender_df = pd.read_csv(dir_path + file)
# tender_df = pd.read_csv("2022_july_2023_april/final_2022_ACCF_26871_1.csv")
    file_schema = open('../../../../Downloads/cdl_assam_tenders_ocds_template.json',encoding='utf-8', errors='ignore')
    data = json.load(file_schema)
    awarded_df = pd.DataFrame(columns = ['Bid Number', 'Bidder Name', 'Awarded Currency', 'Awarded Value',
        'Contract Date :', 'Contract Value :'])
    df_bidders = pd.DataFrame(columns =['S.No.1', 'Bid Number', 'Bidder Name', 'Submitted Date', 'Status',
        'Remarks'])
    main(tender_df,data,awarded_df,df_bidders)

--Return--
None
> /var/folders/pr/cgzvhg3912ndvzh32qyjbt4w0000gn/T/ipykernel_18053/1198104041.py(7)<module>()
      5     if file.endswith(file_extension):
      6         file_list.append(file)
----> 7 pdb.set_trace()
      8 for file in file_list:
      9     tender_df = pd.read_csv(dir_path + file)

ipdb> len(file_list)
39500


In [ ]:
data['awards']